In [ ]:
%%bash
python -V
pip list

In [ ]:
# make sure the tensorflow version is large than 1.8
%%bash
pip list

In [ ]:
%%bash
# example to conver weight
FILE=flower_gpu
tflite_convert \
  --output_file='./weight/mnist.tflite' \
  --keras_model_file='./weight/mnist.h5'
  
python keras_to_tensorflow.py --input_model=weight/${FILE}.h5 --output_model=weight/${FILE}.pb
# toco --graph_def_file='./weight/mnist.pb' --output_file='./weight/mnist.tflite' --input_shape='1,28,28,1' --input_arrays='input' --output_arrays='output/Softmax' --output_format=TFLITE  --input_format=TENSORFLOW_GRAPHDEF 
# at=TENSORFLOW_GRAPHDEF  -inference_type=FlOAT

#should change  Input_layer and Output_layer name to show in keras_to_tensorflow
FILE=cifa 
IMAGE_SIZE=224 \
INPUT=input_2 \
OUTPUT=predictions/Softmax
toco \
  --graph_def_file=weight/${FILE}.pb \
  --output_file=weight/${FILE}.lite \
  --input_format=TENSORFLOW_GRAPHDEF \
  --output_format=TFLITE \
  --input_shape=1,${IMAGE_SIZE},${IMAGE_SIZE},3 \
  --input_array=${INPUT} \
  --output_array=${OUTPUT} \
  --inference_type=FLOAT \
  --input_data_type=FLOAT

### Train a mnist and save .tflite model

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
 
mnist = input_data.read_data_sets("mnist",one_hot=True)
 
 
# 定义批次大小
batch_size = 128
n_batch = mnist.train.num_examples // batch_size
print(n_batch)
 
# 定义placeholder
x = tf.placeholder(tf.float32,[1,784],name='input_x')
y = tf.placeholder(tf.float32,[1,10],name='output_y')
 
# 定义 测试
x_test = tf.placeholder(tf.float32,[None,784],name='input_test_x')
y_test = tf.placeholder(tf.float32,[None,10],name='input_test_y')
 
# 创建一个简单的神经网络
W = tf.Variable(tf.zeros([784,10]),name="W")
b = tf.Variable(tf.zeros([1,10]),name="b")
 
prediction = tf.nn.softmax(tf.matmul(x,W)+b)
 
 
 
# 创建损失函数
train = tf.train.GradientDescentOptimizer(0.02).minimize(tf.reduce_mean(tf.square(y-prediction)))
 
# 名称转换
def canonical_name(x):
  return x.name.split(":")[0]
 
# 计算准确率
test_prediction = tf.nn.softmax(tf.matmul(x_test,W)+b)
accuarcy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y_test,1),tf.argmax(test_prediction,1)),tf.float32))
 
init = tf.global_variables_initializer()
out = tf.identity(prediction, name="output")
 
print('Session')
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(1):
        for batch in range(n_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            for index in range(len(batch_xs)):
                xs = batch_xs[index].reshape(1,784)
                ys = batch_ys[index].reshape(1,10)
                sess.run(train, feed_dict={x: xs, y: ys})
            acc = sess.run(accuarcy,feed_dict={x_test:mnist.test.images,y_test:mnist.test.labels})
            print("over"+str(acc))
 
    frozen_tensors = [out]
    out_tensors = [out]
 
    frozen_graphdef = tf.graph_util.convert_variables_to_constants(sess, sess.graph_def, list(map(canonical_name, frozen_tensors)))
    tflite_model = tf.contrib.lite.toco_convert(frozen_graphdef, [x], out_tensors)
 
    open("writer_model.tflite", "wb").write(tflite_model)


## MNIST Keras

In [ ]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import plot_model
from keras.applications.resnet50 import ResNet50
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from keras import backend as K
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

batch_size = 128
num_classes = 10
epochs = 5

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print(y_train.shape, y_train[0])

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

input = Input(name ='input', shape = input_shape)
x = Conv2D(32, kernel_size=(3, 3),
                 activation='relu')(input)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(num_classes, activation='softmax', name = 'output')(x)
model = Model(inputs = input, outputs = x)


model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
model.summary()
model.save('./weight/mnist.h5')
with open('./weight/mnist.json', 'w+') as f:
    f.write(model.to_json())
plot_model(model, to_file='./weight/mnist.png', show_shapes = True)
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
os.exit()

## Cifar Keras

In [ ]:
img_width, img_height = 224, 224
train_data_dir = '/home/sober/ftp_bk/Dataset/Tmp_Train/cifar/Train/'
val_data_dir = '/home/sober/ftp_bk/Dataset/Tmp_Train/cifar/Val/'
batch_size = 8
epochs = 200
nb_train_samples, nb_validation_samples = 5000,  1000


def mean(x):
    return (x-128.0)/128.0

model = ResNet50(include_top = True, classes = 10, weights=None, input_shape=(img_width, img_height, 3))
# model.summary()
# model.layers.pop()
# output = model.layers[-1].output
# output = Flatten()(output)
# output = Dense(10, activation='softmax')(output)
# model = Model(model.input, output)


model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(lr=0.001),
              metrics=['accuracy'])


    
train_datagen = ImageDataGenerator(horizontal_flip=True,
    vertical_flip=True, rotation_range=15, preprocessing_function=mean)
test_datagen = ImageDataGenerator(preprocessing_function=mean)
train_generator = train_datagen.flow_from_directory(
            train_data_dir, shuffle=True,
            target_size=(img_width, img_height),
            batch_size=batch_size, class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(
            val_data_dir, shuffle=True,
            target_size=(img_width, img_height),
            batch_size=batch_size, class_mode='categorical')

# model.fit(x_train, y_train,
#           batch_size=batch_size,
#           epochs=epochs,
#           verbose=1,
#           validation_data=(x_test, y_test))
# EarlyStopping(monitor='val_acc', patience=5,
#                                       verbose=0,
#                                       mode='auto'),
# ReduceLROnPlateau(monitor='val_acc', factor=0.2,
#                                               patience=10, verbose=0,
#                                               cooldown=1, epsilon=0.001)

callbacks = []
historys = model.fit_generator(callbacks=callbacks,
                                                use_multiprocessing=True,
                                                workers=5, max_queue_size=20,
                                                generator=train_generator,
                                                steps_per_epoch=nb_train_samples // batch_size,
                                                epochs=epochs,
                                                validation_steps=nb_validation_samples // batch_size,
                                                validation_data=validation_generator)
# model.summary()
model.save('./weight/cifa.h5')
with open('./weight/cifa.json', 'w+') as f:
    f.write(model.to_json())
plot_model(model, to_file='./weight/cifa.png', show_shapes = True)

## Keras Model Predict 

In [ ]:
# coding=utf-8

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import shutil
import os
import datetime
import time
import logging
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.models import model_from_json
from os import listdir
from LoadWriteModel import loadWriteModel
from os.path import join, isdir
import gc

plt.switch_backend('agg')


class MyPredict(object):
    # directory
    CATEGORY = 20
    # file
    NORMAL = 10

    def __init__(self, model, labels=None, image_shape=(224, 224), level=None,
                 mother_path='./Predict'):
        """
        :param model:
        :param labels:
        :param image_shape:
        :param level:   [console level] or ['console level', 'log level']
        logging.INFO
        :param mother_path:     log and predict summary image save file path
        """

        self.mother_path = mother_path
        self.labels = labels
        self.image_shape = image_shape
        self.__init_default_variable()
        self.__init_environment()
        self.__init_log(
            level=level if level is not None else self.default_level)
        self.model = model
        logging.info('Init finish')

    def __init_default_variable(self):
        """
        the default variable and path
        :return:
        """
        self.default_logname = 'MyPredict_log.log'
        self.default_level = [logging.DEBUG]
        self.default_pred_filename_ext = '_predict.csv'
        self.default_confusion_matrix_path = join(self.mother_path,
                                                  'confusion_matrix.csv')
        self.default_imagefile_extension = 'jpg|jpeg|bmp|png'

    def __init_environment(self):
        # log remove
        for root, _, files in os.walk('./'):
            for f in files:
                if f.endswith(r'.log'):
                    os.remove(join(root, f))

        # image save path clean
        if os.path.exists(self.mother_path):
            shutil.rmtree(self.mother_path)

        # create mother folder
        os.makedirs(self.mother_path)

    def __init_log(self, level):
        """
        :param level:  level[0] is log file output level, level[1] is console output level
        :return:
        """
        if len(level) == 1:
            if type(level) is list:
                level = level[0]
            logging.basicConfig(level=level)
        elif len(level) == 2:
            logging.basicConfig(filename=self.default_logname, level=level[1])
            console = logging.StreamHandler()
            console.setLevel(level[0])
            logging.getLogger('').addHandler(console)
        else:
            raise RuntimeError("The log level's length must be 1 or 2")
        logging.info('Log init')

    # def set_output_level(self, level):
    #     size = len(level)
    #     assert (size == 1 or size == 2),  'Level length must be 0 or 1'
    #     if size == 1:
    #         level = level[0] if type(level) is list
    #         logging.Logger.setLevel(level)
    #     else:
    #         console.setLevel(level[1])
    #         log

    def set_labels(self, labels):
        """
        :param labels:
        :return:
        """
        if self.labels is None or self.labels == '':
            logging.info("Label Change\n Origin: %s\n Now: %s\n", self.labels,
                         labels)
        self.labels = labels

    def __del__(self):
        self.label = None
        del self.model
        self.model = None
        gc.collect()


    def __init_csv(self):
        """
        init the log file
        :return:
        """
        columns_name = ['file', 'r_label', 'r_index', 't_index', 't_label',
                        'timestamp', 'image', 'check']
        for index in self.labels:
            columns_name.append('l_' + index)
        columns_name = sorted(columns_name)
        self.csv = pd.DataFrame(columns=columns_name)

    @staticmethod
    def read_log_model(name, history_path, history_load=False):
        """

        :param name:
        :param history_path:
        :return:
        """
        """
        read from history file and generate model name if not given
        :param name:    model name which need to be load
        :param history: history path
        :param model:   model
        :return:    loaded model
        """
        assert history_path != None and os.path.exists(
            history_path), 'History path ' \
                           'must be valid'
        assert name != None
        loadboostrap = loadWriteModel(history_path)
        model, _ = loadboostrap.load(name, weight_load=True,
                                     history_load=history_load)
        model.name = name
        return model

    @staticmethod
    def read_file_model(model_path, weight_path, show_summary=False):
        """ read model and load weight from json and h5 file

        model_path = './Result/model/i_shift.json'
        weight_path = './Result/weight/i_shift.h5'
        model = MyPredict.read_file_model(model_path, weight_path, show_summary=True)

        :param model_path:
        :param weight_path:
        :param show_summary:    show model summary or not
        :return:
        """
        start = time.time()
        assert (os.path.exists(model_path) is True and model_path.endswith(
            '.json')), 'Model Path invalid'
        assert (os.path.exists(weight_path) is True and weight_path.endswith(
            '.h5')), 'Weight Path invalid'
        with open(model_path, 'r') as f:
            model = model_from_json(f.read())
        model.load_weights(weight_path, by_name=True)
        end = time.time()
        print('Model Loaded Spend:%0.3fs' % (end - start))
        if show_summary:
            model.summary()

        if model.name is None:
            model.name = MyPredict.generate_name()
        return model

    # def __read_model(self, name=None, history=None, model=None):
    #     """
    #     read from history file and generate model name if not given
    #     :param name:    model name which need to be load
    #     :param history: history path
    #     :param model:   model
    #     :return:    loaded model
    #     """
    #     if model != None:
    #         self.model_name = self.__generate_name()
    #         assert model != None
    #         return model
    #     else:
    #         assert history != None
    #         assert name != None
    #         self.model_name = name
    #         # model load
    #         loadboostrap = loadWriteModel(history)
    #         model, _ = loadboostrap.load(name, weight_load=True)
    #         return model

    def dir_paths(self, path):
        """
        get the validate directory full path from val_path
        :param path:
        :return: directory name and directory path, which has been sorted
        """
        dir_array = []
        for f in listdir(path):
            total_path = join(path, f)
            if isdir(total_path):
                dir_array.append(total_path)
        dir_path_array = sorted(dir_array)
        labels = [os.path.split(d)[-1] for d in dir_path_array]
        return labels, dir_path_array

    def __generate_array(self, path):
        """
        read each image file and save them to a numpy array
        :param path:
        :return: image_arrays, predict_list
        """
        image_arrays = None
        predict_list = []
        if os.path.isdir(path):
            file_list = [join(path, f) for f in listdir(path)]
        else:
            file_list = [path]

        for absolute_path in file_list:
            if self.__check_extension(absolute_path):
                predict_list.append(absolute_path)
                image = load_img(absolute_path)
                image = image.resize(self.image_shape)
                # load all image to numpy and convert to (None, 224, 224, 3)
                image = img_to_array(image)
                image = image[np.newaxis, :]
                try:
                    image_arrays = np.concatenate((image_arrays, image), 0)
                except:
                    image_arrays = image

        if len(predict_list) == 0:
            raise RuntimeError(
                "No image find in predit folder path, Please check the file "
                "extension is " + self.default_imagefile_extension)

        logging.info('Stack image array size: %s', image_arrays.shape)
        return image_arrays, predict_list

    @staticmethod
    def generate_name():
        now = datetime.datetime.now()
        timestamp = now.strftime('%Y%m%d_%H_%M_%S')
        name = timestamp + '_%02d' % (np.random.randint(100))
        return name

    def __predict(self, image_arrays, predict_list, target_index,
                  save_image=True):
        """
        predict, and get result which is a softmax array, and get the max result to compare
            the target one
        :param predict_list:    the predict file list
        :param target_index:    the image which belong to his category
        :param save_image:      save image which is predict wrong
        :return:
        """
        result = self.model.predict(image_arrays)

        # result summary csv row
        for i in range(len(result)):
            real_index = np.argmax(result[i])
            row = {}
            wrong_file = predict_list[i]
            row['file'] = wrong_file
            row['t_index'] = int(target_index)
            row['t_label'] = self.labels[target_index]
            row['r_index'] = int(real_index)
            row['r_label'] = self.labels[real_index]
            name = MyPredict.generate_name()
            row['timestamp'] = name[:-3]
            for index, label in enumerate(self.labels):
                key = 'l_' + label
                row[key] = str(np.round(result[i][index], 3))

            if real_index != target_index:
                # predict wrong
                row['check'] = 0
                logging.debug('Predict:\n%s', row)

                # TODO 保存文件,用统一Flag
                if save_image:
                    row['image'] = self.__save_image(image_arrays[i], result[i],
                                                     wrong_file, name)
            else:
                row['check'] = 1

            self.csv = self.csv.append(row, ignore_index=True)

        logging.info('One Folder image predict finish')

    def __save_image(self, img_array, result, title, file_name):
        """
        save image
        :param img_array:   image array
        :param result:      softmax rate
        :param title:       image title
        :param file_name:   image file name without extension
        :return:
        """

        def auto_label(rects, aix):
            # Get y-axis height to calculate label position from.
            (y_bottom, y_top) = aix.get_ylim()
            y_height = y_top - y_bottom

            for rect in rects:
                height = rect.get_height()

                # Fraction of axis height taken up by this rectangle
                p_height = (height / y_height)

                # If we can fit the label above the column, do that;
                # otherwise, put it inside the column.
                if p_height > 0.95:  # arbitrary; 95% looked good to me.
                    label_position = height - (y_height * 0.05)
                else:
                    label_position = height + (y_height * 0.01)
                height = '%s' % (np.round(height, 2))

                aix.text(rect.get_x() + rect.get_width() / 2., label_position,
                         height, ha='center', va='bottom', fontsize=6)


        labels = self.labels

        plt.figure()
        image = array_to_img(img_array)
        plt.subplot(211)
        plt.title(title)
        plt.axis('off')
        plt.imshow(image)

        # TODO 每个bar都有不同颜色
        ax = plt.subplot(212)
        label_size = len(result)
        xticks = np.arange(label_size)
        bar = plt.bar(xticks, result, align='center', width=0.9)

        # avoid the label size too large
        if label_size > 20:
            labels = [i[:3] for i in labels]
            rotation = 'vertical'
            fontsize = 8
        else:
            labels = tuple(labels)
            rotation = 30
            fontsize = 6
        plt.xticks(xticks, labels, fontsize=fontsize, rotation=rotation)
        auto_label(aix=ax, rects=bar)

        plt.grid()
        plt.tight_layout()
        save_path = join(self.mother_path, file_name + '.jpg')
        plt.savefig(save_path)
        plt.close()
        return save_path

    def __check_extension(self, path):
        assert os.path.exists(path), 'Path %s is not exit !' % path
        if path.endswith(os.sep):
            path = path[:-1]
        f = os.path.split(path)[1]
        if re.match(r'([\w]+\.(?:' + self.default_imagefile_extension + '))',
                    f):
            logging.debug('Check: %s, %s', "True", path)
            return True
        else:
            logging.debug('Check: %s, %s', "False", path)
            return False

    def run(self, pred_path, predict_type, pred_filename=None, ext=None,
            confusion_matrix=True, save_predict_image=True,
            pandas_format=False):

        if ext is not None and ext != '':
            self.default_imagefile_extension = ext

        assert os.path.exists(pred_path), "Predict Path {} is not exist"%(predict_path)


        # just one image
        if predict_type == MyPredict.NORMAL:
            image_arrays, predict_list = self.__generate_array(pred_path)
            results = self.model.predict(image_arrays)

            # without label use [0,.....n] instead
            self.label = self.labels if self.labels != None else [str(index) for
                                                                  index in
                                                                  np.arange(0,
                                                                            len(
                                                                                results[
                                                                                    0]))]

            df = None
            if pandas_format:
                columns = self.label + ['index']
                df = pd.DataFrame(columns=columns)
            for index, result in enumerate(results):
                np.set_printoptions(linewidth=500, suppress=True)
                print('MESSAGE:%s, \t\n%s' % (predict_list[index], result))

                if pandas_format:
                    result_list = list(result.copy())
                    result_list.append(str(predict_list[index]))
                    df = df.append({key: value for key, value in
                                    zip(df.columns.values, result_list)},
                                   ignore_index=True)

                if save_predict_image:
                    self.__save_image(image_arrays[0], result, pred_path,
                                      MyPredict.generate_name())



            if pandas_format:
                return df

        # one folder has many folder each folder's name is label
        elif predict_type == MyPredict.CATEGORY:
            labels, dir_paths = self.dir_paths(pred_path)
            self.labels = labels
            logging.debug('labels: %s, \ndir_paths: %s', labels, dir_paths)

            # log
            if pred_filename is None or pred_filename == '':
                predict_logfile_name = self.model.name + self.default_pred_filename_ext
            else:
                assert pred_filename.endswith('.csv')
                predict_logfile_name = pred_filename

            self.__init_csv()

            for i, path in enumerate(dir_paths):
                logging.info('Program read next directory: %s', path)
                # read every directory's image file and generate image array
                images, predict_list = self.__generate_array(path)
                self.__predict(images, predict_list, i, save_image=save_predict_image)

            # csv save
            self.csv.to_csv(join(self.mother_path, predict_logfile_name),
                            index=False)


            logging.info('Logging Saved')

            if confusion_matrix:
                self.__generate_confusion_matrix()

            if pandas_format:
                return self.csv

        # TODO 预测准确度
        else:
            raise RuntimeError("Predict type must be NORMAL or CATEGORY")

    def __generate_confusion_matrix(self):
        matrix = np.zeros((len(self.labels),) * 2)
        column_name = ['r_index', 't_index']
        search_csv = self.csv[column_name]
        np.set_printoptions(linewidth=500)
        for index, row in search_csv.iterrows():
            i, j = row[column_name[0]], row[column_name[1]]
            matrix[int(i)][int(j)] += 1

        confus_matrix = pd.DataFrame(matrix, index=self.labels,
                                     columns=self.labels, dtype='int64')
        pd.set_option('display.width', 500)
        logging.info("MESSAGE: \n%s" % (confus_matrix))

        confus_matrix.to_csv(self.default_confusion_matrix_path)
        logging.info('Confusion Matrix Saved')


if __name__ == '__main__':
    model_path = './weight/cifa.json'
    weight_path = './weight/cifa.h5'
    predict_path = './cifa'
    # predict_path = '/Users/sober/Workspace/Python/PredictTools/Image/0491.jpg'
    model = MyPredict.read_file_model(model_path, weight_path,
                                      show_summary=True)
#     model = MyPredict.read_log_model(name='xx', history_path='history')
    predict = MyPredict(model=model, level=[logging.INFO], mother_path='xx')
    predict.run(pred_filename='time.csv', predict_type=MyPredict.CATEGORY,
                pred_path=predict_path)


In [ ]:
%matplotlib inline
from keras.models import load_model
import numpy as np
from PIL import Image
import os
from matplotlib.pyplot import imshow
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = ''

def reformat(input):
    output = []
    print(type(input))
    for i in input:
       output.append('%.3f' %i)
    return output

model = load_model('weight/flower_gpu.h5')
filepath = '/home/sober/ftp_bk/Dataset/Tmp_Train/flo/'
for d in os.listdir(filepath):
    dirpath = filepath + d + '/'
    for f in os.listdir(dirpath):
        file = dirpath + f
        print(file)
        im = Image.open(file)
        im = im.resize((150, 150), Image.ANTIALIAS)
        im = np.asarray(im)
        im = im[np.newaxis, :]
#         imshow(np.asarray(im))
#         break
        print(np.array2string(model.predict(np.asarray(im)), formatter={'float_kind':lambda x: "%.2f" % x}))

## TFLite file Test

In [ ]:
import numpy as np
import tensorflow as tf
from PIL import Image
import os
filepath='./weight/lar_mobilenet.lite'
# filepath='./weight/flower5.lite'
val_path = '/home/sober/ftp_bk/Dataset/thesis_data/Val/'
shape=(224, 224)
max_predict = 10
def pre_processing(image):
    '''
        image pre_processing
    '''
    return image / 255.0

# Load TFLite model and allocate tensors.
interpreter = tf.contrib.lite.Interpreter(model_path=filepath)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test model on random input data.
input_shape = input_details[0]['shape']
# change the following line to feed into your own data.


for d in os.listdir(val_path):
    dirpath = val_path+ d + '/'
    for f in os.listdir(dirpath):
        if max_predict < 0:
            exit(0)
        else:
            max_predict -= 1
        
        file = dirpath + f
        print(file)
        im = Image.open(file)
        im = im.resize(shape, Image.ANTIALIAS)
        im = np.asarray(im)
        im = pre_processing(im)
        im = im[np.newaxis, :]
        im = np.asarray(im)
#         print(np.array2string(model.predict(np.asarray(im)), formatter={'float_kind':lambda x: "%.2f" % x}))
        input_data = np.array(im, dtype=np.float32)
        interpreter.set_tensor(input_details[0]['index'], input_data)

        interpreter.invoke()
        output_data = interpreter.get_tensor(output_details[0]['index'])
#         output_data.sort()
        print(np.array2string(output_data, formatter={'float_kind':lambda x: "%.2f" % x}))
    
"daisy dandelion roses sunflowers tulips"

## Tensorflow .pb, .tflite, get input and output layer

In [ ]:
from tensorflow.python.framework import graph_util
from tensorflow.python.platform import gfile
import tensorflow as tf
from keras.models import load_model
import os

def save_graph_to_file(sess,  graph_file_name, output_names):
    output_graph_def = graph_util.convert_variables_to_constants(
      sess,  sess.graph.as_graph_def(),  output_names)
    with gfile.FastGFile(graph_file_name, 'wb') as f:
        f.write(output_graph_def.SerializeToString())

def keras2tflite(sess, input_tensors, output_tensors, filename):
    '''
    input_tensor and output_tensors is array and is keras input and output tensor
    filename must be .tflite
    '''
    converter = tf.contrib.lite.TocoConverter.from_session(sess, input_tensors, output_tensors)
    tflite_model=converter.convert()
    open(filename, "wb").write(tflite_model)
    
def get_input_output_layer(filename) :
    """
    It must clean the gpu memory, otherwise the input and output name is incorrect !
    """
    if 'pb' in filename:
        gf = tf.GraphDef()
        gf.ParseFromString(open(filename,'rb').read())
        for n in gf.node:
            print(n.name + ' =>> ' + n.op )
    elif 'h5' in filename:
        assert(os.path.exists(filename))
        model = load_model(filename)
        input_names = [node.op.name for node in model.inputs]
        output_names = [node.op.name for node in model.outputs]
        print('Input :'  + str(input_names) + '\nOutput: ' +  str(output_names))
        
get_input_output_layer('./weight/cifa.h5')

## Keras Self-build Model seems Cat/Dog

In [ ]:
from keras.models import Sequential, Model
from keras.models import load_model
import tensorflow as tf
from keras import backend as K
from keras.layers import Conv2D, MaxPooling2D, Input
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import os
from keras import optimizers
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

train_path='/home/sober/code/jupyter/ftp_code/AndroidTensorflow/tensorflow-for-poets-2/tf_files/flower_photos'
val_path='/home/sober/code/jupyter/ftp_code/AndroidTensorflow/tensorflow-for-poets-2/tf_files/flower_photos'
batch_size = 16
epochs = 200

input = Input(shape = (150, 150, 3), name = 'input')
x = Conv2D(32, (3, 3), activation='relu')(input)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu')(x)
x = Conv2D(32, (3, 3), activation='relu')(x)
x = Conv2D(32, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
# x = Dense(64, activation='relu')(x)
# x = Dense(2, activation='sigmoid', name = 'output')(x)
x = Dense(5, activation='softmax', name = 'output')(x)
model = Model(inputs = input, outputs= x)

sgd = optimizers.SGD(lr=0.000005, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])



# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator()

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        train_path,  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        val_path,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical')

model.fit_generator(
        train_generator,
        steps_per_epoch=1340 // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        shuffle=True,
        validation_steps=64 // batch_size)
model.save('./weight/flower5.h5')  # always save your weights after training or during training

# convert
output_names = [node.op.name for node in model.outputs]

export_dir = './weight/'
sess = K.get_session()
save_graph_to_file(sess,  export_dir + "flower5.pb", output_names)

